In [35]:
import matplotlib.pyplot as plt
import numpy as np
import casperfpga
import time
from lwa_f import snap2_fengine
from lwa_f import blocks
import pandas as pd
from cr_functions import *
import struct
import math
from scipy import special as sp
from scipy import optimize as oz
from scipy import stats as st
from lwa_antpos import mapping

## Connect to all boards

In [36]:
brdnames=['snap01','snap02','snap03','snap04','snap05','snap06','snap07','snap08','snap09','snap10','snap11']
casperbrds = [casperfpga.CasperFpga(brdname, transport=casperfpga.TapcpTransport) for brdname in brdnames]
fpgfile='/home/ubuntu/kp/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2023-03-23_1851.fpg'


## Set up the ethernet on all boards and then send a single trigger to one board
All the boards should read out their data. I repeated this test many times with different packet wait times.

In [37]:

packetwait=100 #clock cycles between packets
wait_after_readout=200 #time [clock cycles] to wait for other boards to finish readout
for i,brdname in enumerate(brdnames):
    casperbrd=casperbrds[i]
    #prepare to send a snapshot
    
    setup_data_source(casperbrd,"adc") #set the data source to ADC
    setup_ethernet(brdname,casperbrd,fpgfile,'lwacr',packetwait) #set up the ethernet packetizer and ethernet block
    setvalue(casperbrd,'wait_after_readout','cr_registers.xlsx',wait_after_readout) #set the wait time after readout before new internally-generated triggers are accepted
    setvalue(casperbrd,'brd_id','cr_registers.xlsx',int(brdname[-2:])) #set the board id
    setvalue(casperbrd,'eth_enable','cr_registers.xlsx',1)  #enable ethernet


Data source set to adc
------------------------
snap01:cosmic_ray_cr_forty_gbe configuration:
MAC:  02:02:02:01:01:01
Gateway:  0.0.0.1
IP:  10.41.0.201
Fabric port: 
11111
Fabric interface is currently: Enabled
	base_ip: 255.255.255.255
	ip_mask: 255.255.255.255
	rx_ips: []
ARP Table: 
IP: 10.41.0. 58: MAC: 04:3F:72:DF:C2:F8
Setting wait between packets to 100
Data source set to adc
------------------------
snap02:cosmic_ray_cr_forty_gbe configuration:
MAC:  02:02:02:02:02:02
Gateway:  0.0.0.1
IP:  10.41.0.202
Fabric port: 
11111
Fabric interface is currently: Enabled
	base_ip: 255.255.255.255
	ip_mask: 255.255.255.255
	rx_ips: []
ARP Table: 
IP: 10.41.0. 58: MAC: 04:3F:72:DF:C2:F8
Setting wait between packets to 100
Data source set to adc
------------------------
snap03:cosmic_ray_cr_forty_gbe configuration:
MAC:  02:02:02:03:03:03
Gateway:  0.0.0.1
IP:  10.41.0.203
Fabric port: 
11111
Fabric interface is currently: Enabled
	base_ip: 255.255.255.255
	ip_mask: 255.255.255.255
	rx_ips:

In [38]:
#send trigger
software_trigger(casperbrds[3],1)

Packetizer formed 64 packets
Board successfully returned to listening state after readout
Readout took 137669 clock cycles
Packetizer never overflowed
Ethernet block almost-full counter incremented by 0


In [26]:
704*200*10

1408000

## Send repeated triggers to test data rate
### First, send repeat triggers from software.

In [15]:
#check packet and trigger stats before starting
packet_count1=np.asarray([getvalue(brd,"eof_and_valid","cr_registers.xlsx") for brd in casperbrds])
trigger_count1=np.asarray([getvalue(brd,"count_triggers","cr_registers.xlsx") for brd in casperbrds])

#try repeated triggers from software. It turns out not to be fast enough.
t_run=10
t_start=time.time()
t_since_start=time.time()-t_start
count=0
while t_since_start<t_run:
    count+=1
    software_trigger(casperbrds[3],0)  #the stats argument is False because by not reading various stats registers it can go faster
    t_since_start=time.time()-t_start
print(count)

#check packet and trigger statistics after finishing
packet_count2=np.asarray([getvalue(brd,"eof_and_valid","cr_registers.xlsx") for brd in casperbrds])
trigger_count2=np.asarray([getvalue(brd,"count_triggers","cr_registers.xlsx") for brd in casperbrds])

#report summary stats
newtriggers=trigger_count2-trigger_count1
newpackets=packet_count2-packet_count1
print(newtriggers) 
print(newpackets)
print('total packets ',np.sum(newpackets))
if (np.min(newpackets)!=np.max(newpackets)):
    print("Uh oh! Different boards did different things!")


107
[107 107 107 214 107 107 107 107 107 107 107]
[6848 6848 6848 6848 6848 6848 6848 6848 6848 6848 6848]
total packets  75328


In [16]:
704*110

77440


### Run Test with internally-generated triggers

In [27]:
[setvalue(casperbrd,"enable_coinc_trig","cr_registers.xlsx",0) for casperbrd in casperbrds]


[None, None, None, None, None, None, None, None, None, None, None]

In [28]:
#define parameters
trigger_power_thresh=511**2
veto_power_thresh=511**2
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[62]=1
veto_roles_array=np.zeros(64,dtype=int)
antenna_number_thresh=0
veto_number_thresh=0
trigger_window=10
veto_window=10




#set up the coincidencer on ONE board only
casperbrd=casperbrds[3]
setup_data_source(casperbrd,"longcounter") #set the data source to long counter

setup_coincidencer(casperbrd,trigger_power_thresh,
                          veto_power_thresh,
                          trigger_window,
                          veto_window,
                          antenna_number_thresh,
                          veto_number_thresh,
                          core_roles_array,
                          veto_roles_array)

Data source set to longcounter


In [34]:

#check packet and trigger stats before starting
packet_count1=np.asarray([getvalue(brd,"eof_and_valid","cr_registers.xlsx") for brd in casperbrds])
trigger_count1=np.asarray([getvalue(brd,"count_triggers","cr_registers.xlsx") for brd in casperbrds])

#do test
setvalue(casperbrd,"enable_coinc_trig","cr_registers.xlsx",1)
time.sleep(10)
setvalue(casperbrd,"enable_coinc_trig","cr_registers.xlsx",0)

#check packet and trigger statistics after finishing
packet_count2=np.asarray([getvalue(brd,"eof_and_valid","cr_registers.xlsx") for brd in casperbrds])
trigger_count2=np.asarray([getvalue(brd,"count_triggers","cr_registers.xlsx") for brd in casperbrds])

#report summary stats
newtriggers=trigger_count2-trigger_count1
newpackets=packet_count2-packet_count1
print(newtriggers) 
print(newpackets)
print('total packets ',np.sum(newpackets))
if (np.min(newpackets)!=np.max(newpackets)):
    print("Uh oh! Different boards did different things!")


[1874 1874 1874 3748 1874 1874 1874 1874 1874 1874 1874]
[119936 119936 119936 119936 119936 119936 119936 119936 119936 119936
 119936]
total packets  1319296


### Internal triggers generated by all boards at once

In [63]:
#define parameters
trigger_power_thresh=511**2
veto_power_thresh=511**2
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[62]=1
veto_roles_array=np.zeros(64,dtype=int)
antenna_number_thresh=0
veto_number_thresh=0
trigger_window=10
veto_window=10




#set up the coincidencer on ALL boards
[setup_data_source(brd,"longcounter") for brd in casperbrds] #set the data source to long counter

[setup_coincidencer(brd,trigger_power_thresh,
                          veto_power_thresh,
                          trigger_window,
                          veto_window,
                          antenna_number_thresh,
                          veto_number_thresh,
                          core_roles_array,
                          veto_roles_array) for brd in casperbrds]



#check packet and trigger stats before starting
packet_count1=np.asarray([getvalue(brd,"eof_and_valid","cr_registers.xlsx") for brd in casperbrds])
trigger_count1=np.asarray([getvalue(brd,"count_triggers","cr_registers.xlsx") for brd in casperbrds])

#do test
[setvalue(brd,"enable_coinc_trig","cr_registers.xlsx",1) for brd in casperbrds]
time.sleep(10)
[setvalue(brd,"enable_coinc_trig","cr_registers.xlsx",0) for brd in casperbrds]

#check packet and trigger statistics after finishing
packet_count2=np.asarray([getvalue(brd,"eof_and_valid","cr_registers.xlsx") for brd in casperbrds])
trigger_count2=np.asarray([getvalue(brd,"count_triggers","cr_registers.xlsx") for brd in casperbrds])

#report summary stats
newtriggers=trigger_count2-trigger_count1
newpackets=packet_count2-packet_count1
print(newtriggers) 
print(newpackets)
print('total packets ',np.sum(newpackets))
if (np.min(newpackets)!=np.max(newpackets)):
    print("Uh oh! Different boards did different things!")


Data source set to longcounter
Data source set to longcounter
Data source set to longcounter
Data source set to longcounter
Data source set to longcounter
Data source set to longcounter
Data source set to longcounter
Data source set to longcounter
Data source set to longcounter
Data source set to longcounter
Data source set to longcounter
[3872 3873 3872 3873 3873 3872 3872 3872 3872 3872 3872]
[124928 124928 124928 124928 124928 124928 124928 124928 124928 124928
 124928]
total packets  1374208


In [66]:
print(np.mean(newtriggers)/2)

1936.1363636363637
